# Text to speech Google API

## Required libraries

In [107]:
#!pip install --upgrade google-cloud-texttospeech
#!pip install --upgrade google-cloud-storage
#!pip install cloudstorage 
from google.cloud import texttospeech
from google.oauth2 import service_account
import IPython.display as ipd
import html
from google.datalab import Context
import google.datalab.storage as storage

## Google service account credentials

In [108]:
file_credentials = "[your_credentials].json"
credentials = service_account.Credentials. from_service_account_file(file_credentials)

## Plain text to SSML

In [109]:
def text_to_ssml(text):
    # Generates SSML text from plaintext
    # Arguments: inputfile - string name of plaintext file
    # Returns:A string of SSML text based on plaintext input

# Replace special characters with HTML Ampersand Character Codes
    # These Codes prevent the API from confusing text with
    # SSML commands
    # For example, '<' --> '&lt;' and '&' --> '&amp;'

    escaped_lines = html.escape(text)

    # Convert plaintext to SSML
    # Wait two seconds between each address
    ssml = "<speak><prosody>{}</prosody></speak>".format(
        escaped_lines.replace("\n", '\n<break time="400ms"/>')
    )
    
    return ssml

## Voice specifications

In [110]:
voice = texttospeech.types.VoiceSelectionParams(
        language_code="en-GB",
        name="en-GB-Wavenet-A",
        ssml_gender=texttospeech.enums.SsmlVoiceGender.FEMALE,
    )

## Text to speech

In [111]:
def synthesize_text(example_text, voice):

    client = texttospeech.TextToSpeechClient(credentials=credentials)
    
    synthesis_input = texttospeech.types.SynthesisInput(ssml=example_text)

    print(synthesis_input)
    
    audio_config = texttospeech.types.AudioConfig(
        audio_encoding=texttospeech.enums.AudioEncoding.LINEAR16,
        speaking_rate=0.90,
        pitch=-2
    )

    print(audio_config)

    response = client.synthesize_speech(
        synthesis_input, voice, audio_config
    )

    with open('output.wav', 'wb') as out:
    # Write the response to the output file.
        out.write(response.audio_content)
        print('Audio content written to file "output.wav"')
    out.close()

## Read text file from Google Cloud Storage Bucket

In [112]:
text_in_gcs='example.txt'   # Text file to text-to-speech function

In [113]:
project_id=Context.default().project_id
folder_bucket_name = project_id + '-ttsfiles'
folder_bucket = storage.Bucket(folder_bucket_name)

if not folder_bucket.exists():
    folder_bucket.create()

In [114]:
#Upload the example.txt file in the folder_bucket in GPC Console 

text_file=folder_bucket.object(text_in_gcs)

contents=text_file.read_stream().decode('utf-8')
print(contents)

In Twintig, we want to be an active part of the construction industry innovation.
Construction plays a key role in building the future of the modern world.
It employs about 7 percent of the world’s working-age population, and is one of the world economy’s largest sectors.


## Run text to speech function

In [115]:
example_text = text_to_ssml(contents)
synthesize_text(example_text, voice)

ssml: "<speak><prosody>In Twintig, we want to be an active part of the construction industry innovation.\r\n<break time=\"400ms\"/>Construction plays a key role in building the future of the modern world.\r\n<break time=\"400ms\"/>It employs about 7 percent of the world\342\200\231s working-age population, and is one of the world economy\342\200\231s largest sectors.</prosody></speak>"

audio_encoding: LINEAR16
speaking_rate: 0.9
pitch: -2.0

Audio content written to file "output.wav"


## Listening!

In [116]:
ipd.Audio("output.wav")

## Copy audio file to GCP Bucket

In [117]:
!gsutil cp output.wav gs://[your_project_id]-ttsfiles/output2.wav

Copying file://output.wav [Content-Type=audio/x-wav]...
/ [1 files][815.8 KiB/815.8 KiB]                                                
Operation completed over 1 objects/815.8 KiB.                                    


## Play the GCP Bucket file just created

In [106]:
audio_file=folder_bucket.object("output2.wav")
ipd.Audio(audio_file.read_stream())